In [2]:
import pandas as pd
import numpy as np
import torch

import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from cleaning.pipeline import strategic_fill, shift_forecast_columns

In [3]:
project_root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
data_path = os.path.join(project_root, 'data', 'raw_data', 'france_household_consumption.csv')
print(f"Trying to load from: {data_path}")
raw_data = pd.read_csv(data_path, sep=';', low_memory=False)

# Inspect the first few rows to understand the data format
print("First 5 rows of the raw data:")
print(raw_data.head())

# Check data types
print("\nData types:")
print(raw_data.dtypes)

# Fuse Date and Time columns into a single datetime column
# The format appears to be DD/MM/YYYY for Date and HH:MM:SS for Time
print("\nCreating datetime column...")
raw_data['datetime'] = pd.to_datetime(raw_data['Date'] + ' ' + raw_data['Time'], 
                                     format='%d/%m/%Y %H:%M:%S',
                                     errors='coerce')

# Set the timezone to France (Western European Time)
# Handle daylight saving time transitions by explicitly setting ambiguous times to the first occurrence
# This fixes the AmbiguousTimeError for times like 2007-10-28 01:00:00 that occur twice during DST fallback
raw_data['datetime'] = raw_data['datetime'].dt.tz_localize('Europe/Paris', 
                                                          nonexistent='shift_forward',
                                                          ambiguous=True)  # Use True to select the first occurrence

# Check if there were any parsing errors
null_dates = raw_data['datetime'].isnull().sum()
print(f"Number of null datetime values: {null_dates}")

# If needed, we can drop the original Date and Time columns
# raw_data.drop(['Date', 'Time'], axis=1, inplace=True)

# Set the datetime column as the index
# raw_data.set_index('datetime', inplace=True)

print("\nSample of data with new datetime column:")
print(raw_data[['Date', 'Time', 'datetime']].head())

print(raw_data.columns)

# raw_data['time'] = pd.to_datetime(raw_data['time'], utc=True)
# raw_data.set_index('time', inplace=True)

Trying to load from: /Users/aryanahri/epf_datasets/data/raw_data/france_household_consumption.csv
First 5 rows of the raw data:
         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0  



In [4]:
# Remove the 'Date' and 'Time' columns
raw_data.drop(['Date', 'Time'], axis=1, inplace=True)

# Rename the 'datetime' column to 'time'
raw_data.rename(columns={'datetime': 'time'}, inplace=True)

# Verify the changes
print("\nColumns after removing Date/Time and renaming datetime:")
print(raw_data.columns)
print("\nSample of data with renamed time column:")
print(raw_data[['time', 'Global_active_power']].head())



Columns after removing Date/Time and renaming datetime:
Index(['Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3', 'time'],
      dtype='object')

Sample of data with renamed time column:
                       time Global_active_power
0 2006-12-16 17:24:00+01:00               4.216
1 2006-12-16 17:25:00+01:00               5.360
2 2006-12-16 17:26:00+01:00               5.374
3 2006-12-16 17:27:00+01:00               5.388
4 2006-12-16 17:28:00+01:00               3.666


In [5]:
# Set the 'time' column as the index of the DataFrame
raw_data.set_index('time', inplace=True)

# Verify that 'time' is now the index
print("\nDataFrame after setting 'time' as index:")
print(raw_data.head())
print("\nIndex name:", raw_data.index.name)
print("Index type:", type(raw_data.index))



DataFrame after setting 'time' as index:
                          Global_active_power Global_reactive_power  Voltage  \
time                                                                           
2006-12-16 17:24:00+01:00               4.216                 0.418  234.840   
2006-12-16 17:25:00+01:00               5.360                 0.436  233.630   
2006-12-16 17:26:00+01:00               5.374                 0.498  233.290   
2006-12-16 17:27:00+01:00               5.388                 0.502  233.740   
2006-12-16 17:28:00+01:00               3.666                 0.528  235.680   

                          Global_intensity Sub_metering_1 Sub_metering_2  \
time                                                                       
2006-12-16 17:24:00+01:00           18.400          0.000          1.000   
2006-12-16 17:25:00+01:00           23.000          0.000          1.000   
2006-12-16 17:26:00+01:00           23.000          0.000          2.000   
2006-12-16 17:27:

In [6]:
start_time = raw_data.index.min()
end_time = raw_data.index.max()
complete_range = pd.date_range(start=start_time, end=end_time, freq='min')

# Find missing hours and then insert these times to make the time consecutive
missing_hours = complete_range.difference(raw_data.index)
print(missing_hours)
missing_df = pd.DataFrame(index=missing_hours)
raw_data = pd.concat([raw_data, missing_df])
raw_data = raw_data.sort_index()

filled_df = strategic_fill(raw_data)

DatetimeIndex(['2007-10-28 02:00:00+01:00', '2007-10-28 02:01:00+01:00',
               '2007-10-28 02:02:00+01:00', '2007-10-28 02:03:00+01:00',
               '2007-10-28 02:04:00+01:00', '2007-10-28 02:05:00+01:00',
               '2007-10-28 02:06:00+01:00', '2007-10-28 02:07:00+01:00',
               '2007-10-28 02:08:00+01:00', '2007-10-28 02:09:00+01:00',
               ...
               '2010-10-31 02:50:00+01:00', '2010-10-31 02:51:00+01:00',
               '2010-10-31 02:52:00+01:00', '2010-10-31 02:53:00+01:00',
               '2010-10-31 02:54:00+01:00', '2010-10-31 02:55:00+01:00',
               '2010-10-31 02:56:00+01:00', '2010-10-31 02:57:00+01:00',
               '2010-10-31 02:58:00+01:00', '2010-10-31 02:59:00+01:00'],
              dtype='datetime64[ns, Europe/Paris]', length=240, freq=None)


In [7]:
filled_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
2006-12-16 17:24:00+01:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
2006-12-16 17:25:00+01:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2006-12-16 17:26:00+01:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
2006-12-16 17:27:00+01:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
2006-12-16 17:28:00+01:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [8]:
null_indices = filled_df.index[filled_df.isnull().any(axis=1)]
print("Indices with null values:")
print(null_indices)

print("\nColumns at each index with null values:")
for idx in null_indices:
    print(f"\nIndex: {idx}")
    print(filled_df.loc[idx])

Indices with null values:
DatetimeIndex([], dtype='datetime64[ns, Europe/Paris]', freq=None)

Columns at each index with null values:


In [9]:
# Hour of the Day (0 to 23)
filled_df['HoD'] = filled_df.index.hour
# Day of the Week (1=Monday to 7=Sunday)
filled_df['DoW'] = filled_df.index.dayofweek + 1
filled_df['MoY'] = filled_df.index.month

In [10]:
filled_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,HoD,DoW,MoY
2006-12-16 17:24:00+01:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0,17,6,12
2006-12-16 17:25:00+01:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0,17,6,12
2006-12-16 17:26:00+01:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0,17,6,12
2006-12-16 17:27:00+01:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0,17,6,12
2006-12-16 17:28:00+01:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0,17,6,12


In [ ]:
# # Save the cleaned data to a CSV file
# filled_df.to_csv(f'{project_root}/data/cleaned/cleaned_france_data.csv')
# print("Data saved to '../data/processed/france_data_cleaned.csv'")


Data saved to '../data/processed/france_data_cleaned.csv'


In [12]:
print(len(filled_df))

2075499


In [13]:
row_idx = 0
print(f"Date at index {row_idx}: {filled_df.index[row_idx]}")

Date at index 0: 2006-12-16 17:24:00+01:00


In [14]:
row_idx = 1245299
print(f"Date at index {row_idx}: {filled_df.index[row_idx]}")

Date at index 1245299: 2009-04-29 10:23:00+02:00


In [15]:
row_idx = 1660399
print(f"Date at index {row_idx}: {filled_df.index[row_idx]}")

Date at index 1660399: 2010-02-11 15:43:00+01:00


In [16]:
row_idx = 2075498
print(f"Date at index {row_idx}: {filled_df.index[row_idx]}")

Date at index 2075498: 2010-11-26 21:02:00+01:00


In [17]:
# Check for non-numeric values in each column of the dataframe
non_numeric_columns = []

for column in filled_df.columns:
    # Skip checking the index which is the datetime column
    if not pd.api.types.is_numeric_dtype(filled_df[column]):
        non_numeric_columns.append(column)
        print(f"Column '{column}' contains non-numeric values")
        # Display a sample of non-numeric values
        non_numeric_values = filled_df[~pd.to_numeric(filled_df[column], errors='coerce').notna()][column]
        if not non_numeric_values.empty:
            print(f"Sample non-numeric values: {non_numeric_values.head().tolist()}")

if not non_numeric_columns:
    print("All columns contain numeric values only.")


Column 'Global_active_power' contains non-numeric values
Sample non-numeric values: ['?', '?', '?', '?', '?']
Column 'Global_reactive_power' contains non-numeric values
Sample non-numeric values: ['?', '?', '?', '?', '?']
Column 'Voltage' contains non-numeric values
Sample non-numeric values: ['?', '?', '?', '?', '?']
Column 'Global_intensity' contains non-numeric values
Sample non-numeric values: ['?', '?', '?', '?', '?']
Column 'Sub_metering_1' contains non-numeric values
Sample non-numeric values: ['?', '?', '?', '?', '?']
Column 'Sub_metering_2' contains non-numeric values
Sample non-numeric values: ['?', '?', '?', '?', '?']


In [18]:
# Replace '?' values with NaN in the entire dataframe
print("Checking for '?' values in the dataframe...")

# Count the number of '?' values in each column before replacement
question_mark_counts = {}
for column in filled_df.columns:
    question_mark_count = (filled_df[column] == '?').sum()
    if question_mark_count > 0:
        question_mark_counts[column] = question_mark_count

if question_mark_counts:
    print("Found '?' values in the following columns:")
    for column, count in question_mark_counts.items():
        print(f"  - {column}: {count} occurrences")
    
    # Replace '?' with NaN
    filled_df = filled_df.replace('?', np.nan)
    print("Replaced all '?' values with NaN")
    
    # Verify replacement
    remaining_question_marks = 0
    for column in filled_df.columns:
        remaining_question_marks += (filled_df[column] == '?').sum()
    
    if remaining_question_marks == 0:
        print("Verification successful: No '?' values remain in the dataframe")
    else:
        print(f"Warning: {remaining_question_marks} '?' values still remain in the dataframe")
else:
    print("No '?' values found in the dataframe")


Checking for '?' values in the dataframe...
Found '?' values in the following columns:
  - Global_active_power: 25979 occurrences
  - Global_reactive_power: 25979 occurrences
  - Voltage: 25979 occurrences
  - Global_intensity: 25979 occurrences
  - Sub_metering_1: 25979 occurrences
  - Sub_metering_2: 25979 occurrences
Replaced all '?' values with NaN
Verification successful: No '?' values remain in the dataframe


In [19]:
start_time = raw_data.index.min()
end_time = raw_data.index.max()
complete_range = pd.date_range(start=start_time, end=end_time, freq='min')

# Find missing hours and then insert these times to make the time consecutive
missing_hours = complete_range.difference(filled_df.index)
print(missing_hours)
missing_df = pd.DataFrame(index=missing_hours)
filled_df = pd.concat([filled_df, missing_df])
filled_df = filled_df.sort_index()

filled_df = strategic_fill(filled_df)

DatetimeIndex([], dtype='datetime64[ns, Europe/Paris]', freq='min')


In [20]:
# Save the cleaned data to a CSV file
filled_df.to_csv(f'{project_root}/data/cleaned/cleaned_france_data.csv')
print("Data saved to '../data/processed/france_data_cleaned.csv'")


Data saved to '../data/processed/france_data_cleaned.csv'
